<a href="https://colab.research.google.com/github/adammoss/bnn_hmc/blob/main/results/MLSST_MCD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install astro-datasets --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for astro-datasets: filename=astro_datasets-0.0.9-py3-none-any.whl size=15986 sha256=82bd24371dac6de3a7f5feb8c36dd7a503065642a11e3aa3c3e6bbca35dc91f5
  Stored in directory: /root/.cache/pip/wheels/77/11/df/7eb516aa333b47883b39adabbe38508a09163be741b476dcda
Successfully built astro-datasets


In [2]:
!pip install tensorflow_datasets --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import tensorflow as tf
import numpy as np
import os
import astro_datasets
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

In [4]:
ds_train_Y10, info_train_Y10 = tfds.load(name='mlsst/Y10', split='train', with_info=True, as_supervised=True, batch_size=50)
ds_valid_Y10, info_valid_Y10 = tfds.load(name='mlsst/Y10', split='validation', with_info=True, as_supervised=True, batch_size=50)
ds_test_Y10, info_test_Y10 = tfds.load(name='mlsst/Y10', split='test', with_info=True, as_supervised=True, batch_size=50)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

...

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling ~/tensorflow_datasets/mlsst/Y10/1.0.0.incompleteMY6GJ5/mlsst-validation.tfrecord*...:   0%|         …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling ~/tensorflow_datasets/mlsst/Y10/1.0.0.incompleteMY6GJ5/mlsst-test.tfrecord*...:   0%|          | 0/6…

Dataset mlsst downloaded and prepared to ~/tensorflow_datasets/mlsst/Y10/1.0.0. Subsequent calls will reuse this data.


In [5]:
#for d in ds_train_Y10.take(2):
  #inputs, label = d
  #print(label)
  #plt.figure(figsize=(6,6))
  #plt.imshow(tf.math.sinh(inputs[0,:,:,0]))
  #plt.show()

In [6]:

def normalize(image, label):  
  image = tf.math.asinh(image)
  return image, label



ds_train_Y10 = ds_train_Y10.map(normalize)
ds_valid_Y10 = ds_valid_Y10.map(normalize)
ds_test_Y10 = ds_test_Y10.map(normalize)

In [7]:
class MonteCarloDropout(tf.keras.layers.Dropout):
     def call(self, inputs):
         return super().call(inputs, training=True)

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(6, kernel_size=5, strides=1, padding = 'same', input_shape=(100, 100, 3)),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(1, 1)),
    MonteCarloDropout(0.1),
    tf.keras.layers.Conv2D(16, kernel_size=5, strides=1, padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(1, 1)),
    MonteCarloDropout(0.1),
    tf.keras.layers.Conv2D(26, kernel_size=5, strides=1, padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(1, 1)),
    MonteCarloDropout(0.1),
    tf.keras.layers.Conv2D(32, kernel_size=5, strides=1, padding = 'same'),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPool2D(pool_size=(1, 1)),
    tf.keras.layers.Flatten(),
    MonteCarloDropout(0.1),
    tf.keras.layers.Dense(120),
    tf.keras.layers.ReLU(),
    MonteCarloDropout(0.1),
    tf.keras.layers.Dense(84),
    tf.keras.layers.ReLU(),
    MonteCarloDropout(0.1),
    tf.keras.layers.Dense(3, activation='softmax'),
    ])

In [9]:
#model.summary()

In [10]:
loss=tf.keras.losses.SparseCategoricalCrossentropy()

In [11]:
opt = tf.keras.optimizers.Adam(learning_rate=5e-5)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.95, patience=3)

In [12]:
model.compile(optimizer=opt,
              loss=loss,
              metrics='accuracy')

In [13]:
checkpoint_path = "mlsst/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                  save_weights_only=False,
                                                  monitor='val_loss',
                                                  mode='min',
                                                  verbose=1,
                                                  save_best_only=True)

In [14]:
model.fit(ds_train_Y10, epochs=100, callbacks=[reduce_lr, cp_callback], validation_data=ds_valid_Y10)

Epoch 1/100
470/470 [==============================] - ETA: 0s - loss: 1.0658 - accuracy: 0.5051
Epoch 1: val_loss improved from inf to 0.92909, saving model to mlsst/cp.ckpt
INFO:tensorflow:Assets written to: mlsst/cp.ckpt/assets


INFO:tensorflow:Assets written to: mlsst/cp.ckpt/assets


470/470 [==============================] - 64s 105ms/step - loss: 1.0658 - accuracy: 0.5051 - val_loss: 0.9291 - val_accuracy: 0.5586 - lr: 5.0000e-05
Epoch 2/100
470/470 [==============================] - ETA: 0s - loss: 0.8756 - accuracy: 0.5777
Epoch 2: val_loss improved from 0.92909 to 0.90225, saving model to mlsst/cp.ckpt
INFO:tensorflow:Assets written to: mlsst/cp.ckpt/assets


INFO:tensorflow:Assets written to: mlsst/cp.ckpt/assets


470/470 [==============================] - 82s 172ms/step - loss: 0.8756 - accuracy: 0.5777 - val_loss: 0.9022 - val_accuracy: 0.5696 - lr: 5.0000e-05
Epoch 3/100
470/470 [==============================] - ETA: 0s - loss: 0.7718 - accuracy: 0.6341
Epoch 3: val_loss improved from 0.90225 to 0.87346, saving model to mlsst/cp.ckpt
INFO:tensorflow:Assets written to: mlsst/cp.ckpt/assets


...

470/470 [==============================] - 63s 133ms/step - loss: 0.7718 - accuracy: 0.6341 - val_loss: 0.8735 - val_accuracy: 0.5860 - lr: 5.0000e-05
Epoch 4/100
470/470 [==============================] - ETA: 0s - loss: 0.6538 - accuracy: 0.6937
Epoch 4: val_loss improved from 0.87346 to 0.85353, saving model to mlsst/cp.ckpt
INFO:tensorflow:Assets written to: mlsst/cp.ckpt/assets


INFO:tensorflow:Assets written to: mlsst/cp.ckpt/assets


470/470 [==============================] - 82s 175ms/step - loss: 0.6538 - accuracy: 0.6937 - val_loss: 0.8535 - val_accuracy: 0.6310 - lr: 5.0000e-05
Epoch 5/100
470/470 [==============================] - ETA: 0s - loss: 0.5756 - accuracy: 0.7445
Epoch 5: val_loss improved from 0.85353 to 0.84816, saving model to mlsst/cp.ckpt
INFO:tensorflow:Assets written to: mlsst/cp.ckpt/assets


INFO:tensorflow:Assets written to: mlsst/cp.ckpt/assets


470/470 [==============================] - 63s 133ms/step - loss: 0.5756 - accuracy: 0.7445 - val_loss: 0.8482 - val_accuracy: 0.6244 - lr: 5.0000e-05
Epoch 6/100
470/470 [==============================] - ETA: 0s - loss: 0.5046 - accuracy: 0.7777
Epoch 6: val_loss did not improve from 0.84816
470/470 [==============================] - 43s 91ms/step - loss: 0.5046 - accuracy: 0.7777 - val_loss: 0.8731 - val_accuracy: 0.6516 - lr: 5.0000e-05
Epoch 7/100
470/470 [==============================] - ETA: 0s - loss: 0.4611 - accuracy: 0.8017
Epoch 7: val_loss did not improve from 0.84816
470/470 [==============================] - 42s 88ms/step - loss: 0.4611 - accuracy: 0.8017 - val_loss: 0.9101 - val_accuracy: 0.6420 - lr: 5.0000e-05
Epoch 8/100
470/470 [==============================] - ETA: 0s - loss: 0.4064 - accuracy: 0.8316
Epoch 8: val_loss did not improve from 0.84816
470/470 [==============================] - 41s 88ms/step - loss: 0.4064 - accuracy: 0.8316 - val_loss: 0.9603 - val_a

In [15]:
new_model = tf.keras.models.load_model('mlsst/cp.ckpt')

In [16]:
for _ in range(10):
  correct, n = 0, 0
  for (x, y) in ds_test_Y10:
    yhat = new_model.predict(x)
    correct += np.sum(np.argmax(yhat, axis=-1) == y)
    n += len(y)
  print('Accuracy:',correct / n)

Accuracy: 0.6218912881608339
Accuracy: 0.6184661206254654
Accuracy: 0.6238272524199553
Accuracy: 0.6172747580044676
Accuracy: 0.6159344750558451
Accuracy: 0.6098287416232315
Accuracy: 0.6148920327624721
Accuracy: 0.6008935219657483
Accuracy: 0.6151898734177215
Accuracy: 0.6086373790022338


In [17]:
samples_iter = 200
softmax = []

for _ in range(samples_iter):
  s = []
  for (x, y) in ds_test_Y10:
    pred = new_model.predict(x)
    #softmax.append(pred)
    s.append(pred)
  s = np.concatenate(s)
  softmax.append(s)

softmax = np.array(softmax)



def entropy_MI(softmax, samples_iter):

    class_0 = softmax[:][0]
    class_1 = softmax[:][1]
    class_2 = softmax[:][2]
    #biased estimator of predictive entropy, bias will reduce as samples_iter is increased
    predictive_entropy = -((np.sum(class_0)/samples_iter) * np.log(np.sum(class_0)/samples_iter) + (np.sum(class_1)/samples_iter) * np.log(np.sum(class_1)/samples_iter)
                          + (np.sum(class_2)/samples_iter) * np.log(np.sum(class_2)/samples_iter))
    
    #Orignal: -((np.sum(class_0)/samples_iter) * np.log(np.sum(class_0)/samples_iter) + (np.sum(class_1)/samples_iter) * np.log(np.sum(class_1)/samples_iter))
    

    mutual_info = predictive_entropy + np.sum((class_0 * np.log(class_0)) + (class_1 * np.log(class_1)) + (class_2 * np.log(class_2)))/samples_iter

    #Original: predictive_entropy + np.sum(class_0*np.log(class_0))/samples_iter +  np.sum(class_1*np.log(class_1))/samples_iter
    
    #entropy of a single pass
    class0 = softmax[:][0][0]
    class1 = softmax[:][1][0]
    class2 = softmax[:][2][0]
    entropy_singlepass = -np.sum((class0 * np.log(class0)) + (class1 * np.log(class1)) + (class2 * np.log(class2)))/samples_iter

    #Original: -(class0 * np.log(class0) + class1 * np.log(class1))

    #print("Entropy of a single pass:", entropy_singlepass)
    return predictive_entropy, mutual_info, entropy_singlepass



predictive_entropy, mutual_info, entropy_singlepass = entropy_MI(softmax, samples_iter)


print("Predictive Entropy:", predictive_entropy)
print("Mutual Information:", mutual_info)
print("Entropy of a single pass:", entropy_singlepass)

Predictive Entropy: -353.92566600417695
Mutual Information: -423.47857127761444
Entropy of a single pass: 0.012489269971847534


In [18]:
ds_test_Y1, info_test_Y1 = tfds.load(name='mlsst/Y1', split='test', with_info=True, as_supervised=True, batch_size=50)
ds_test_Y1 = ds_test_Y1.map(normalize)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

...

Generating validation examples...: 0 examples [00:00, ? examples/s]

Shuffling ~/tensorflow_datasets/mlsst/Y1/1.0.0.incomplete5VINJY/mlsst-validation.tfrecord*...:   0%|          …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling ~/tensorflow_datasets/mlsst/Y1/1.0.0.incomplete5VINJY/mlsst-test.tfrecord*...:   0%|          | 0/67…

Dataset mlsst downloaded and prepared to ~/tensorflow_datasets/mlsst/Y1/1.0.0. Subsequent calls will reuse this data.


In [19]:
for _ in range(10):
  correct, n = 0, 0
  for (x, y) in ds_test_Y1:
    yhat = new_model.predict(x)
    correct += np.sum(np.argmax(yhat, axis=-1) == y)
    n += len(y)
  print('Accuracy:',correct / n)

Accuracy: 0.46835443037974683
Accuracy: 0.4673119880863738
Accuracy: 0.4674609084139985
Accuracy: 0.4673119880863738
Accuracy: 0.47163067758749067
Accuracy: 0.46120625465376025
Accuracy: 0.4695457930007446
Accuracy: 0.4732688011913626
Accuracy: 0.46552494415487716
Accuracy: 0.47505584512285925


In [20]:
samples_iter = 200
softmax = []

for _ in range(samples_iter):
  s = []
  for (x, y) in ds_test_Y1:
    pred = new_model.predict(x)
    #softmax.append(pred)
    s.append(pred)
  s = np.concatenate(s)
  softmax.append(s)

softmax = np.array(softmax)


predictive_entropy, mutual_info, entropy_singlepass = entropy_MI(softmax, samples_iter)


print("Predictive Entropy:", predictive_entropy)
print("Mutual Information:", mutual_info)
print("Entropy of a single pass:", entropy_singlepass)

Predictive Entropy: -353.925654984202
Mutual Information: -417.660010452952
Entropy of a single pass: 0.012511255741119385
